In [1]:
# Prérequis :
# - les fiches métiers au format JSON dans un dossier fiches_metiers
# - l'arborescence des activités dans un dossier ArborescencesActivitesCompetences
# - l'arborescence des compétences dans un dossier ArborescenceActivitesCompetences
# - un fichier activites.json dans un dossier fichier_donnees
# - un fichier competences.json dans un dossier fichier_donnees

# Ce script permet de créer un fichier csv avec :
# - les métiers en ligne
# - des indicatrices pour les compétences et les activités en colonnes
import os
import json
import pandas as pd
import numpy as np

In [3]:
# Préparation des chemins utiles
dir_path = os.getcwd()
fiches_dir_path = os.path.join(dir_path, 'fiches_metiers')
arbo_act = os.path.join(dir_path,'ArborescencesActivitesCompetences/rome_arboactivités_juin62181.xlsx')
arbo_comp = os.path.join(dir_path,'ArborescencesActivitesCompetences/rome_arbocompétences_juin62182.xlsx')
act_json = os.path.join(dir_path,'fichier_donnees/activites.json')
comp_json = os.path.join(dir_path,'competences.json')
output_file = os.path.join(dir_path,'PE_niv23.csv')

In [3]:
# Liste des métiers
list_files = os.listdir(fiches_dir_path)
n = len(list_files)

In [5]:
# Chargement des données de compétences dans la variable comp
with open(comp_json) as json_data:
    comp = json.load(json_data)
    json_data.close()
    
# Chargement des données de d'activités dans la variable act
with open(act_json) as json_data:
    act = json.load(json_data)
    json_data.close()

In [36]:
# Création de la liste de toutes les compétences de niveau 3 features_comp_niv3
features_comp_niv3 = []
for niv1 in comp:
    for niv2 in comp[niv1]:
        for niv3 in comp[niv1][niv2]:
            if niv3=='nan':
                features_comp_niv3.append(niv2)
            else:
                features_comp_niv3.append(niv3)

                
# Création de la liste de toutes les activités de niveau 2 features_act
features_act_niv2 = []
for niv1 in act:
    for niv2 in act[niv1]:
        features_act_niv2.append(niv2)

# Création de la liste de toutes les caracéristiques de niveau 2
features_niv23 = features_comp_niv3 + features_act_niv2

In [76]:
# Création d'un dataframe qui contiendra les futures valeurs des caractéristiques des métiers
features_head = ['code_rome', 'intitule']
cols = features_head + features_niv23
p = len(cols)

data = pd.DataFrame(np.zeros(shape=(n,p)), columns = cols)

In [75]:
data.columns

Index(['code_rome', 'intitule', 'Légal', 'Fiscal', 'Environnement',
       'Sécurité, Protection des biens et des personnes', 'Normes',
       'Procédure de conditionnement, stockage',
       'Procédures de montage, réglage, entretien',
       'Procédures administratives',
       ...
       'Prospecter une clientèle, un partenaire',
       'Transporter des marchandises à l'aide d'un véhicule, livrer',
       'Manipuler un outil, un appareil, un instrument',
       'Conduire une machine', 'Conduire un véhicule, un engin',
       'Porter une charge par ses propres moyens',
       'Transporter des personnes, des animaux', 'Ranger, conditionner, trier',
       'Animer un groupe, une réunion (hors hiérarchie)',
       'Former une personne, enseigner'],
      dtype='object', length=170)

In [58]:
# Fonction qui permet de récupérer une liste des code ogr des activités et compétences
# qui décrvient le métier donné en argument
def get_job_features_ogr(file):
    job_features = []
    with open(file, 'r') as f:
        job = json.load(f)
        
    for item in job["activites_de_base"]:
        job_features.append(int(item['code_ogr']))
    for item in job["savoir_theorique_et_proceduraux"]:
        job_features.append(int(item['code_ogr']))
    for item in job["activite_specifique"]:
        job_features.append(int(item['code_ogr']))
    for item in job["savoir_action"]:
        job_features.append(int(item['code_ogr']))
    return job_features

37

In [77]:
# Cette boucle permet de compléter le dataframe des métiers avec les valeurs de leurs activités
# de niveau 2 et compétences de niveau 3
i = 0
for file in list_files:
    # Ouverture de la fiche métier du job dans un dictionnaire 'job'
    with open(fiches_dir_path + '/' + file, 'r') as f:
        job = json.load(f)
    
    # On renseigne le code rome et l'intitulé du métier dans le dataframe final   
    data.loc[i,'code_rome'] = job["code_rome"]     
    data.loc[i,'intitule'] = job["intitule"]
    
    job_features_ogr = get_job_features_ogr(fiches_dir_path + '/' + file)
    len(job_features_ogr)
    
    # On parcourt la liste des compétences et si elle est dans la description du métier 
    # on l'ajoute dans le décompte des compétences de niveau 2
    count = {}
    for niv1 in comp:
        for niv2 in comp[niv1]:
            count[niv2] = 0
            for niv3 in comp[niv1][niv2]:
                count[niv3] = 0
                for niv4 in comp[niv1][niv2][niv3].values():
                    for item in niv4:         
                        if int(item[1]) in job_features_ogr:
                            if niv3=='nan':
                                count[niv2] += 1  
                            else:
                                count[niv3] += 1   
                data.loc[i,niv3] = count[niv3]    
            data.loc[i,niv2] = count[niv2]
    
    # Même chose pour les activités
    for niv1 in act:
        for niv2 in act[niv1]:
            count[niv2] = 0
            for niv3 in act[niv1][niv2]:
                for niv4 in act[niv1][niv2][niv3]:
                    for niv5 in act[niv1][niv2][niv3][niv4].values():
                        for item in niv5:         
                            if int(item[1]) in job_features_ogr:
                                count[niv2] += 1
            data.loc[i,niv2] = count[niv2]
    i +=1

sorted_data = data.sort_values(by='code_rome')

In [78]:
sorted_data.sum(1)

273    36.0
452    35.0
51     26.0
517    31.0
45     36.0
461    36.0
346    36.0
267    30.0
474    37.0
435    28.0
324    22.0
27     14.0
215    30.0
201    43.0
413    32.0
279    43.0
218    40.0
504    35.0
519    47.0
246    33.0
440    49.0
234    58.0
178    29.0
513    29.0
507    32.0
189    19.0
175    22.0
447    34.0
29     24.0
408    38.0
       ... 
332    44.0
160    43.0
179    38.0
116    21.0
100    61.0
301    24.0
221    45.0
143    43.0
375    27.0
26     32.0
516    59.0
98     49.0
522    48.0
146    40.0
108    36.0
450    42.0
187    59.0
62     28.0
342    41.0
405    28.0
24     57.0
157    36.0
131    33.0
506    29.0
466    32.0
329    27.0
265    23.0
120    19.0
20     35.0
359    22.0
dtype: float64

In [79]:
sorted_data.to_csv(output_file)

In [65]:
## tests

file = '/home/timothee/PycharmProjects/ImpalaPoleEmploi2/fiches_metiers/A1101.xml.json'
# Ouverture de la fiche métier du job dans un dictionnaire
with open(file, 'r') as f:
    job = json.load(f)

# On renseigne le code rome et l'intitulé du métier    
data.loc[0,'code_rome'] = job["code_rome"]     
data.loc[0,'intitule'] = job["intitule"]

job_features_ogr = get_job_features_ogr(file)
len(job_features_ogr)

# On parcourt la liste des compétences et si elle décrit le métier on l'ajoute dans le décompte
# des compétences de niveau 2
count = {}
for niv1 in comp:
    for niv2 in comp[niv1]:
        count[niv2] = 0
        for niv3 in comp[niv1][niv2]:
            count[niv3] = 0
            for niv4 in comp[niv1][niv2][niv3].values():
                for item in niv4:         
                    if int(item[1]) in job_features_ogr:
                        if niv3=='nan':
                            count[niv2] += 1  
                        else:
                            count[niv3] += 1   
            data.loc[0,niv3] = count[niv3]    
        data.loc[0,niv2] = count[niv2]


    # Même chose pour les activités
for niv1 in act:
    for niv2 in act[niv1]:
        count[niv2] = 0
        for niv3 in act[niv1][niv2]:
            for niv4 in act[niv1][niv2][niv3]:
                for niv5 in act[niv1][niv2][niv3][niv4].values():
                    for item in niv5:
                        if int(item[1]) in job_features_ogr:
                            count[niv2] += 1
        data.loc[0,niv2] = count[niv2]

data.sum(1)[0]

36.0

In [48]:
data.iloc[0,0:168]

code_rome                                                                                                            A1101
intitule                                                                         Conduite d'engins agricoles et forestiers
Légal                                                                                                                    0
Fiscal                                                                                                                   0
Environnement                                                                                                            1
Sécurité, Protection des biens et des personnes                                                                          1
Normes                                                                                                                   0
Procédure de conditionnement, stockage                                                                                   0
Procédures de mo

In [43]:
c = {}
c['abcd'] = 0